In [1]:
dataset1 = []

class DataPoint:
  def __init__(self, status, vars):
    self.isTrue = status
    self.title = vars[0]
    self.text = vars[1]
    self.subject = vars[2]
    self.data = vars[3]

  def print(self):
    print(self.isTrue + " - " + self.title + " - " + self.text)
    



In [10]:
import pandas as pd

dfFalse = pd.read_csv (r'Databases/Data1/Fake.csv')
ds1 = pd.DataFrame()
ds1["text"] = dfFalse["text"]
ds1.insert(0, 'label', 'fake')
dfTrue = pd.read_csv (r'Databases/Data1/True.csv')
ds2 = pd.DataFrame()
ds2["text"] = dfTrue["text"]
ds2.insert(0, 'label', 'true')
dataset1 = pd.concat([ds1, ds2])
dataset1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
print(dataset1)


      label                                               text
0      fake  Donald Trump just couldn t wish all Americans ...
1      fake  House Intelligence Committee Chairman Devin Nu...
2      fake  On Friday, it was revealed that former Milwauk...
3      fake  On Christmas day, Donald Trump announced that ...
4      fake  Pope Francis used his annual Christmas Day mes...
5      fake  The number of cases of cops brutalizing and ki...
6      fake  Donald Trump spent a good portion of his day a...
7      fake  In the wake of yet another court decision that...
8      fake  Many people have raised the alarm regarding th...
9      fake  Just when you might have thought we d get a br...
10     fake  A centerpiece of Donald Trump s campaign, and ...
11     fake  Republicans are working overtime trying to sel...
12     fake  Republicans have had seven years to come up wi...
13     fake  The media has been talking all day about Trump...
14     fake  Abigail Disney is an heiress with brass ov

In [3]:
dfTrue2 = pd.read_csv (r'Databases/Data1/True.csv')
ds22 = pd.DataFrame()
ds22["text"] = dfTrue2["text"]
ds22.insert(0, 'label', 'true')
ds22['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [4]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en_core_web_lg
!{sys.executable} -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
LGVars = pd.DataFrame()
import numpy as np
import spacy as sp
import pandas as pd

# Load English tokenizer, tagger, parser and NER
nlp = sp.load("en_core_web_sm")

# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)

cols = ("text", "lemma", "POS", "explain", "stopword")
rows = []
for t in doc:
    row = [t.text, t.lemma_, t.pos_, sp.explain(t.pos_), t.is_stop]
    rows.append(row)
df = pd.DataFrame(rows, columns=cols)
print(df)

         text      lemma    POS                    explain  stopword
0        When       when  SCONJ  subordinating conjunction      True
1   Sebastian  Sebastian  PROPN                proper noun     False
2       Thrun      Thrun  PROPN                proper noun     False
3     started      start   VERB                       verb     False
4     working       work   VERB                       verb     False
5          on         on    ADP                 adposition      True
6        self       self   NOUN                       noun     False
7           -          -  PUNCT                punctuation     False
8     driving      drive   VERB                       verb     False
9        cars        car   NOUN                       noun     False
10         at         at    ADP                 adposition      True
11     Google     Google  PROPN                proper noun     False
12         in         in    ADP                 adposition      True
13       2007       2007    NUM   

In [6]:
import sys
!{sys.executable} -m pip install xgboost

In [12]:
import pandas as pd

dfFalse = pd.read_csv (r'Databases/Data1/Fake.csv')
ds1 = pd.DataFrame()
ds1["text"] = dfFalse["text"]
ds1.insert(0, 'label', 'fake')
dfTrue = pd.read_csv (r'Databases/Data1/True.csv')
ds2 = pd.DataFrame()
ds2["text"] = dfTrue["text"]
ds2.insert(0, 'label', 'true')
dataset1 = pd.concat([ds1, ds2])
dataset1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

# Import libraries
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

print(dataset1.head(5))

nltk.download('stopwords')

def text_preprocess(message):
    # Remove punctuations
    nopunc = [char for char in message if char not in string.punctuation]

    # Join the characters again
    nopunc = "".join(nopunc)
    nopunc = nopunc.lower()

    # Remove any stopwords and characters that arent letters
    nostop = [
        word
        for word in nopunc.split()
        if word.lower() not in stopwords.words("english") and word.isalpha()
    ]
    return nostop


false_text = dataset1[dataset1["label"] == "fake"]["text"]
true_text = dataset1[dataset1["label"] == "true"]["text"]
print(f"Number of fake messages: {len(false_text)}")
print(f"Number of true messages: {len(true_text)}")



  label                                               text
0  fake  Donald Trump just couldn t wish all Americans ...
1  fake  House Intelligence Committee Chairman Devin Nu...
2  fake  On Friday, it was revealed that former Milwauk...
3  fake  On Christmas day, Donald Trump announced that ...
4  fake  Pope Francis used his annual Christmas Day mes...
Number of fake messages: 23452
Number of true messages: 21417


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jjfri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
####################################################################### TAKES FOREVER TO PROSESS
from sklearn.utils import shuffle
data_shuffled = dataset1.sample(frac=1).reset_index(drop=True)
print(data_shuffled.head(10))
#data_shuffled = dataset1.sample(len(dataset1))

# 0.0878 seconds per row // 89.922 min for entire dataset

# Remove punctuations/stopwords from all messages
data_shuffled["text"] = data_shuffled["text"].apply(text_preprocess)
data_shuffled.head(20)



  label                                               text
0  true  BEIJING (Reuters) - China must keep up efforts...
1  fake  Donald Trump likes to promote himself a shrewd...
2  true  JOHANNESBURG (Reuters) - A South African High ...
3  true  WASHINGTON (Reuters) - President Donald Trump ...
4  fake  21st Century Wire says There is an amazing sce...
5  true   NEW YORK (Reuters) - A judge will decide by t...
6  fake  Bill Clinton was falsely confident in his plan...
7  fake  Nassau County cops busted a Long Island man wh...
8  fake  With 28 days left to go until election day, a ...
9  fake  Putin Tells Russian Army To Prepare For WWWIII...


,label,text
0,true,"[beijing, reuters, china, must, keep, efforts,..."
1,fake,"[donald, trump, likes, promote, shrewd, busine..."
2,true,"[johannesburg, reuters, south, african, high, ..."
3,true,"[washington, reuters, president, donald, trump..."
4,fake,"[century, wire, says, amazing, scene, unfoldin..."
5,true,"[new, york, reuters, judge, decide, end, month..."
6,fake,"[bill, clinton, falsely, confident, plan, end,..."
7,fake,"[nassau, county, cops, busted, long, island, m..."
8,fake,"[days, left, go, election, day, new, poll, nbc..."
9,fake,"[putin, tells, russian, army, prepare, wwwiii,..."


In [14]:
# Convert messages (as lists of string tokens) to strings
data_shuffled["text"] = data_shuffled["text"].agg(lambda x: " ".join(map(str, x)))
data_shuffled.head()

# Initialize count vectorizer
vectorizer = CountVectorizer()
bow_transformer = vectorizer.fit(data_shuffled["text"])

# Fetch the vocabulary set
print(f"20 BOW Features: {vectorizer.get_feature_names()[20:40]}")
print(f"Total number of vocab words: {len(vectorizer.vocabulary_)}")

# Convert strings to vectors using BoW
dataset1_bow = bow_transformer.transform(data_shuffled["text"])

# Print the shape of the sparse matrix and count the number of non-zero occurrences
print(f"Shape of sparse matrix: {dataset1_bow.shape}")
print(f"Amount of non-zero occurrences: {dataset1_bow.nnz}")



20 BOW Features: ['aaja', 'aalberg', 'aalberts', 'aaldef', 'aaliyah', 'aalo', 'aamer', 'aamin', 'aaminus', 'aammir', 'aamom', 'aampampacbreturn', 'aampampbcbparsefloatastringbaeffunctionavar', 'aampexkamp', 'aan', 'aapa', 'aapi', 'aapl', 'aaplo', 'aaplus']
Total number of vocab words: 198902
Shape of sparse matrix: (44869, 198902)
Amount of non-zero occurrences: 7113367


In [15]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(dataset1_bow)

# Transform entire BoW into tf-idf corpus
messages_tfidf = tfidf_transformer.transform(dataset1_bow)
print(messages_tfidf.shape)

# Convert spam and ham labels to 0 and 1 (or, vice-versa)
FactorResult = pd.factorize(data_shuffled["label"])
data_shuffled["label"] = FactorResult[0]
data_shuffled.head()

(44869, 198902)


,label,text
0,0,beijing reuters china must keep efforts revolu...
1,1,donald trump likes promote shrewd businessman ...
2,0,johannesburg reuters south african high court ...
3,0,washington reuters president donald trump spea...
4,1,century wire says amazing scene unfolding amer...


In [20]:
# Split the dataset to train and test sets
msg_train, msg_test, label_train, label_test = train_test_split(
    messages_tfidf, data_shuffled["label"], test_size=0.2
)

print(f"train dataset features size: {msg_train.shape}")
print(f"train dataset label size: {label_train.shape}")

print(f"test dataset features size: {msg_test.shape}")
print(f"test dataset label size: {label_test.shape}")

# Train an xgboost classifier
from xgboost import XGBClassifier

# Instantiate our model
clf = XGBClassifier()

# Fit the model to the training data
clf.fit(msg_train, label_train)


# Make predictions
predict_train = clf.predict(msg_train)

print(
    f"Accuracy of Train dataset: {metrics.accuracy_score(label_train, predict_train):0.3f}"
)

# an example prediction
print(
    "predicted:",
    clf.predict(
        tfidf_transformer.transform(bow_transformer.transform([data_shuffled["text"][9]]))
    )[0],
)
print("expected:", data_shuffled["label"][9])

# print the overall accuracy of the model
label_predictions = clf.predict(msg_test)
print(f"Accuracy of the model: {metrics.accuracy_score(label_test, label_predictions):0.3f}")

train dataset features size: (35895, 198902)
train dataset label size: (35895,)
test dataset features size: (8974, 198902)
test dataset label size: (8974,)
Accuracy of Train dataset: 1.000
predicted: 1
expected: 1
Accuracy of the model: 0.997


In [28]:
#clf.save_model(r"C:\Users\jjfri\OneDrive\Documents\code\GitHub\Fake-News-Risk-Identification\CLF_model.txt")
print(clf)
#clf.save_model('./CLF_model.txt')

import pickle
with open("./CLF_model.txt","wb") as f:
    pickle.dump(messages_tfidf, f)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
       colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
       early_stopping_rounds=None, enable_categorical=False,
       eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
       importance_type=None, interaction_constraints='',
       learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
       max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
       missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
       num_parallel_tree=1, objective='binary:logistic', predictor='auto',
       random_state=0, reg_alpha=0, reg_lambda=1,
       sampling_method='uniform', scale_pos_weight=1, subsample=1,
       tree_method='exact', use_label_encoder=False, validate_parameters=1,
       verbosity=None)


OSError: [Errno 9] Bad file descriptor

Below this is unfinished code for predicting new data

(44869, 198902)
(44869, 198902)


In [ ]:
NYTwebsite_text = """On his first presidential visit to Saudi Arabia, Mr. Biden received an unsmiling fist bump from the crown prince before meeting King Salman. Their body language is being parsed amid tensions over the Saudi human rights record. President Biden used his visit to Israel this week to bolster the blossoming relationship between Israel and a handful of Arab states. But it is Mr. Biden’s own relationship with Saudi Arabia that looms largest over the second part of his four-day trip, and the kingdom’s welcome of him was being closely watched as an indication of how the overall visit might unfold.

As Air Force One landed, just before 6 p.m. local time, the Saudi reception, while led by a senior member of the royal family, was decidedly lacking in pomp. Lining the lilac carpet were a small number of uniformed security officers bearing swords, but even fewer than were dispatched to welcome President Barack Obama when he arrived in the kingdom to a chilly reception in 2016.

Mr. Biden emerged from the plane and made his way down the stairs for his first visit to Saudi Arabia as president, wearing his aviator shades, waving and looking serious. He was greeted by Princess Reema bint Bandar Al Saud, the Saudi ambassador to Washington, and Prince Khalid Al Faisal, a senior member of the royal family who is the governor of Mecca and is close to King Salman.

The president climbed into his limousine just two minutes after disembarking. He was  taken directly to meet the king.

Arriving at the al-Salam palace, Mr. Biden was met by Crown Prince Mohammed bin Salman, and the two exchanged an apparently wordless fist bump before the prince led the president inside.

The Saudi-owned Al Arabiya satellite channel broadcast images of Mr. Biden being warmly welcomed by King Salman, and shaking his hand. Other images showed Mr. Biden and the king sitting next to each other in armchairs in an ornate room during a meeting, with other U.S. and Saudi officials, including Prince Mohammed, sitting nearby.

Later, Mr. Biden is to hold a “working session” along with his team with Prince Mohammed and various Saudi ministers.Biden’s arrival in Saudi Arabia was polite but perfunctory compared with the enthusiastic greeting President Donald J. Trump received in 2017 when King Salman welcomed him on the tarmac. Mr. Trump made Saudi Arabia his first overseas destination as president, a sharp break with tradition and a sign of how closely he would align his administration with the Saudis.

Even though Mr. Biden vowed as a candidate to punish Saudi Arabia over Mr. Khashoggi’s murder by making the kingdom a “pariah,” the president decided that it was worth the political cost of traveling there this week to counter Chinese influence, press for additional oil production and encourage closer ties with Israel.

In other cases, recently including Cuba and Venezuela, Mr. Biden has stressed that his administration is making democracy and respect for human rights the paramount consideration for dealing with other nations’ leaders. But on Thursday in Jerusalem, he said he was going to Saudi Arabia in order to promote U.S. interests.

Those include getting the kingdom to pump more oil from its somewhat modest spare capacity."""

NYTwebsite_text1 = '''WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support education, scientific research, infrastructure, public health and environmental protection. “The (Trump) administration has already been willing to say: ‘We’re going to increase non-defense discretionary spending ... by about 7 percent,’” Meadows, chairman of the small but influential House Freedom Caucus, said on the program. “Now, Democrats are saying that’s not enough, we need to give the government a pay raise of 10 to 11 percent. For a fiscal conservative, I don’t see where the rationale is. ... Eventually you run out of other people’s money,” he said. Meadows was among Republicans who voted in late December for their party’s debt-financed tax overhaul, which is expected to balloon the federal budget deficit and add about $1.5 trillion over 10 years to the $20 trillion national debt. “It’s interesting to hear Mark talk about fiscal responsibility,” Democratic U.S. Representative Joseph Crowley said on CBS. Crowley said the Republican tax bill would require the  United States to borrow $1.5 trillion, to be paid off by future generations, to finance tax cuts for corporations and the rich. “This is one of the least ... fiscally responsible bills we’ve ever seen passed in the history of the House of Representatives. I think we’re going to be paying for this for many, many years to come,” Crowley said. Republicans insist the tax package, the biggest U.S. tax overhaul in more than 30 years,  will boost the economy and job growth. House Speaker Paul Ryan, who also supported the tax bill, recently went further than Meadows, making clear in a radio interview that welfare or “entitlement reform,” as the party often calls it, would be a top Republican priority in 2018. In Republican parlance, “entitlement” programs mean food stamps, housing assistance, Medicare and Medicaid health insurance for the elderly, poor and disabled, as well as other programs created by Washington to assist the needy. Democrats seized on Ryan’s early December remarks, saying they showed Republicans would try to pay for their tax overhaul by seeking spending cuts for social programs. But the goals of House Republicans may have to take a back seat to the Senate, where the votes of some Democrats will be needed to approve a budget and prevent a government shutdown. Democrats will use their leverage in the Senate, which Republicans narrowly control, to defend both discretionary non-defense programs and social spending, while tackling the issue of the “Dreamers,” people brought illegally to the country as children. Trump in September put a March 2018 expiration date on the Deferred Action for Childhood Arrivals, or DACA, program, which protects the young immigrants from deportation and provides them with work permits. The president has said in recent Twitter messages he wants funding for his proposed Mexican border wall and other immigration law changes in exchange for agreeing to help the Dreamers. Representative Debbie Dingell told CBS she did not favor linking that issue to other policy objectives, such as wall funding. “We need to do DACA clean,” she said.  On Wednesday, Trump aides will meet with congressional leaders to discuss those issues. That will be followed by a weekend of strategy sessions for Trump and Republican leaders on Jan. 6 and 7, the White House said. Trump was also scheduled to meet on Sunday with Florida Republican Governor Rick Scott, who wants more emergency aid. The House has passed an $81 billion aid package after hurricanes in Florida, Texas and Puerto Rico, and wildfires in California. The package far exceeded the $44 billion requested by the Trump administration. The Senate has not yet voted on the aid. '''


fake_WP = '''Sara Carter has been right on everything she is reported on with the Mueller investigation. She is a gem of a reporter who does the hard work that so many other journalists just don t. She tells why she believes the FBI will have a major shake-up soon there are 27 leakers that the IG is looking at! Yes, 27 leakers!Sara Carter: We re going to see parts of that report before December (end of the month). We re going to see other parts of his report coming out after January. And they re looking at Peter Strzok. They re looking at Comey. They re looking at 27 leakers. It would not surprise me if there was a shake-up at the FBI and a housecleaning.Tom Fitton of Judicial Watch is another incredible researcher and stubborn investigator. Thank goodness for all of the FOIA requests and lawsuits he is filed to force information to come out of the FBI and DOJ. His latest discovery is an explosive email that exposes anti-Trump Mueller operatives:THANK GOODNESS FOR JUDICIAL WATCH! They pushed for emails from the DOJ and got a treasure trove of incriminating evidence of anti-Trump bias. Judicial Watch President Tom Fitton said it best when he said the email is an  astonishing and disturbing find A top prosecutor who is now a deputy for Special Counsel Robert Mueller is Russia probe praised then-acting Attorney General Sally Yates (see more on Yates below) after she was fired in January by President Trump for refusing to defend his controversial travel ban.The email, obtained by Judicial Watch through a federal lawsuit, shows that on the night of Jan. 30, Andrew Weissmann wrote to Yates under the subject line,  I am so proud. He continued,  And in awe. Thank you so much. All my deepest respects. The disclosure follows confirmation that another Mueller investigator, FBI official Peter Strzok, was fired over the summer after allegedly sending anti-Trump texts to an FBI lawyer with whom he was romantically involved.His alleged actions revived concerns about the objectivity of the FBI probes of both Hillary Clinton is email setup and Russia election meddling.Judicial Watch President Tom Fitton called the new Weissmann document an  astonishing and disturbing find. Andrew Weisman, a key prosecutor on Robert Mueller is team, praised Obama DOJ holdover Sally Yates after she lawlessly thwarted President Trump,  he said in a statement.  How much more evidence do we need that the Mueller operation has been irredeemably compromised by anti-Trump partisans? CATHERINE HERRIDGE:DOJ POLITICAL HACK SALLY YATES YOU RE FIRED! The acting Attorney General, Sally Yates, has betrayed the Department of Justice by refusing to enforce a legal order designed to protect the citizens of the United States. This order was approved as to form and legality by the Department of Justice Office of Legal Counsel.Ms. Yates is an Obama Administration appointee who is weak on borders and very weak on illegal immigration.It is time to get serious about protecting our country. Calling for tougher vetting for individuals travelling from seven dangerous places is not extreme. It is reasonable and necessary to protect our country.Tonight, President Trump relieved Ms. Yates of her duties and subsequently named Dana Boente, U.S. Attorney for the Eastern District of Virginia, to serve as Acting Attorney General until Senator Jeff Sessions is finally confirmed by the Senate, where he is being wrongly held up by Democrat senators for strictly political reasons. I am honored to serve President Trump in this role until Senator Sessions is confirmed. I will defend and enforce the laws of our country to ensure that our people and our nation are protected,  said Dana Boente, Acting Attorney General. '''

dict = {'label':['unknown', 'unknown', 'unknown'],
        'text':[NYTwebsite_text, NYTwebsite_text1, fake_WP],
       }
WP = pd.DataFrame(dict)
WP["text"] = WP["text"].apply(text_preprocess)

# Convert messages (as lists of string tokens) to strings
WP["text"] = WP["text"].agg(lambda x: " ".join(map(str, x)))
print(WP.head())

# Initialize count vectorizer
vectiz = CountVectorizer()
bow_trans = vectiz.fit(WP["text"])

# Fetch the vocabulary set
print(f"20 BOW Features: {vectiz.get_feature_names()[20:40]}")
print(f"Total number of vocab words: {len(vectiz.vocabulary_)}")

# Convert strings to vectors using BoW
WP_bow = bow_trans.transform(WP["text"])

# Print the shape of the sparse matrix and count the number of non-zero occurrences
print(f"Shape of sparse matrix: {WP_bow.shape}")
print(f"Amount of non-zero occurrences: {WP_bow.nnz}")

tfidf_trans = TfidfTransformer().fit(WP_bow)

# Transform entire BoW into tf-idf corpus
WP_tfidf = tfidf_trans.transform(WP_bow)

# an example prediction
v = pd.DataFrame(WP["text"])

var = bow_transformer.transform(v)
print(tfidf_transformer.transform(var))
pred_label = clf.predict(tfidf_transformer.transform(var))[0]
print(
    "predicted:",
    pred_label,
)
WP["label"] = pred_label
WP.head()
#print("expected:", data_shuffled.iloc[9]["label"])


     label                                               text
0  unknown  first presidential visit saudi arabia mr biden...
1  unknown  washington reuters head conservative republica...
2  unknown  sara carter right everything reported mueller ...
20 BOW Features: ['among', 'andrew', 'another', 'antitrump', 'apparently', 'appointee', 'approach', 'approve', 'approved', 'arab', 'arabia', 'arabiya', 'armchairs', 'arrival', 'arrivals', 'arrived', 'arriving', 'assist', 'assistance', 'astonishing']
Total number of vocab words: 673
Shape of sparse matrix: (3, 673)
Amount of non-zero occurrences: 730
  (0, 173596)	1.0
predicted: 0


,label,text
0,0,first presidential visit saudi arabia mr biden...
1,0,washington reuters head conservative republica...
2,0,sara carter right everything reported mueller ...


0    morning joe hosts mika brzezinski joe scarboro...
1    washington reuters us house committee investig...
2    campaign trail donald trump former reality sho...
3    children nestled snug beds oops illegal alien ...
4    washington reuters us house representatives se...
5    new york reuters comic gag turned gaffe democr...
6    mexico city reuters mexican president enrique ...
7    jakarta reuters indonesia disaster mitigation ...
8    former fbi informant blew whistle highprofile ...
9    think real moment like nixon trump dogwhistles...
Name: text, dtype: object


ValueError: Found input variables with inconsistent numbers of samples: [8974, 1]

In [ ]:
# an example prediction
for num_to_predict in range(1):
    dict = {'label':['unknown'],
        'text':[data_shuffled["text"][num_to_predict]],
       }
    v = pd.DataFrame(dict)
    print(v)
    
    # Initialize count vectorizer
    vectorizer = CountVectorizer()
    bow_transformer = vectorizer.fit(v["text"])

    # Fetch the vocabulary set
    print(f"20 BOW Features: {vectorizer.get_feature_names()[20:40]}")
    print(f"Total number of vocab words: {len(vectorizer.vocabulary_)}")

    # Convert strings to vectors using BoW
    dataset1_bow = bow_transformer.transform(v["text"])

    # Print the shape of the sparse matrix and count the number of non-zero occurrences
    print(f"Shape of sparse matrix: {dataset1_bow.shape}")
    print(f"Amount of non-zero occurrences: {dataset1_bow.nnz}")
    
    tfidf_transformer = TfidfTransformer().fit(dataset1_bow)

    # Transform entire BoW into tf-idf corpus
    messages_tfidf = tfidf_transformer.transform(dataset1_bow)
    print(messages_tfidf.shape)
    
    print(
        "predicted:",
        clf.predict(messages_tfidf)[0],
    )
    #print("expected:", data_shuffled.iloc[num_to_predict]["label"])